In [3]:
import pandas as pd
import os
import arxiv

import urllib
from sklearn.metrics.pairwise import cosine_similarity
import fitz
import tensorflow_hub as hub
import numpy as np 

In [4]:
import time
import matplotlib as plt

In [33]:
#use tranfer learning
#takes longer to load than dan
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [20]:
table_emergency_copy = pd.read_csv('table_emergency_copy.csv', index_col='Unnamed: 0')

In [22]:
table_emergency_copy

,url,abstract,id_arxiv,abstract_Arxiv
0,https://aclanthology.org/2021.woah-1.2,Hate speech and profanity detection suffer fro...,2106.07505v1,Hate speech and profanity detection suffer fro...
1,https://aclanthology.org/2021.woah-1.3,"We introduce HateBERT, a re-trained BERT model...",2010.12472v1,"In this paper, we introduce HateBERT, a re-tra..."
2,https://aclanthology.org/2021.woah-1.4,Hateful memes pose a unique challenge for curr...,2107.04313v1,Hateful memes pose a unique challenge for curr...
3,https://aclanthology.org/2021.woah-1.5,Content moderation is often performed by a col...,2107.04212v1,Content moderation is often performed by a col...
4,https://aclanthology.org/2021.woah-1.10,Bias mitigation approaches reduce models{'} de...,2108.01721v1,Bias mitigation approaches reduce models' depe...
...,...,...,...,...
7915,https://aclanthology.org/C16-1333,"We propose a novel semantic tagging task, semt...",1609.07053v1,"We propose a novel semantic tagging task, semt..."
7916,https://aclanthology.org/2016.iwslt-1.5,In this paper we provide the largest published...,1610.01108v1,In this paper we provide the largest published...
7917,https://aclanthology.org/2016.iwslt-1.6,"In this paper, we present our first attempts i...",1611.04798v1,"In this paper, we present our first attempts i..."
7918,https://aclanthology.org/2016.amta-researchers.4,The utilization of statistical machine transla...,1610.04265v1,The utilization of statistical machine transla...


In [30]:
cop = table_emergency_copy.copy()
cop = cop.reset_index()

ran = cop.loc[:9,['abstract', 'url']]
ran2 = cop.loc[len(table_emergency_copy)-10:len(table_emergency_copy),['abstract_Arxiv', 'id_arxiv']]
ran2 = ran2.reset_index()
random = ran.join(ran2)
# random.loc[[3],['title_Arxiv', 'pdf_url_Arxiv', 'published_Arxiv', 'result_Arxiv']] = ran2

In [31]:
random

,abstract,url,index,abstract_Arxiv,id_arxiv
0,Hate speech and profanity detection suffer fro...,https://aclanthology.org/2021.woah-1.2,7910,Although more additional corpora are now avail...,1607.08693v1
1,"We introduce HateBERT, a re-trained BERT model...",https://aclanthology.org/2021.woah-1.3,7911,Target-dependent sentiment classification rema...,1512.01100v1
2,Hateful memes pose a unique challenge for curr...,https://aclanthology.org/2021.woah-1.4,7912,Using neural networks to generate replies in h...,1607.00970v1
3,Content moderation is often performed by a col...,https://aclanthology.org/2021.woah-1.5,7913,Recurrent Neural Network (RNN) is one of the m...,1611.06639v1
4,Bias mitigation approaches reduce models{'} de...,https://aclanthology.org/2021.woah-1.10,7914,We study the role of the second language in bi...,1607.05014v1
5,Automatic detection of toxic language plays an...,https://aclanthology.org/2021.woah-1.12,7915,"We propose a novel semantic tagging task, semt...",1609.07053v1
6,Community-level bans are a common tool against...,https://aclanthology.org/2021.woah-1.18,7916,In this paper we provide the largest published...,1610.01108v1
7,Text simplification is the process of splittin...,https://aclanthology.org/2021.wnut-1.1,7917,"In this paper, we present our first attempts i...",1611.04798v1
8,We present the first openly available corpus f...,https://aclanthology.org/2021.wnut-1.3,7918,The utilization of statistical machine transla...,1610.04265v1
9,Multimodal named entity recognition (MNER) req...,https://aclanthology.org/2021.wnut-1.11,7919,"In this paper, we propose an effective way for...",1607.01628v1


In [34]:
random['cosine_abstract'] = random.apply(lambda x:cosine_similarity(embed([x.abstract]),embed([x.abstract_Arxiv])), axis=1)
random['cosine_abstract'] = random['cosine_abstract'].astype(float)



In [42]:
random[['url','abstract','id_arxiv', 'abstract_Arxiv','cosine_abstract']]

,url,abstract,id_arxiv,abstract_Arxiv,cosine_abstract
0,https://aclanthology.org/2021.woah-1.2,Hate speech and profanity detection suffer fro...,1607.08693v1,Although more additional corpora are now avail...,0.452752
1,https://aclanthology.org/2021.woah-1.3,"We introduce HateBERT, a re-trained BERT model...",1512.01100v1,Target-dependent sentiment classification rema...,0.445323
2,https://aclanthology.org/2021.woah-1.4,Hateful memes pose a unique challenge for curr...,1607.00970v1,Using neural networks to generate replies in h...,0.418310
3,https://aclanthology.org/2021.woah-1.5,Content moderation is often performed by a col...,1611.06639v1,Recurrent Neural Network (RNN) is one of the m...,0.397419
4,https://aclanthology.org/2021.woah-1.10,Bias mitigation approaches reduce models{'} de...,1607.05014v1,We study the role of the second language in bi...,0.329843
5,https://aclanthology.org/2021.woah-1.12,Automatic detection of toxic language plays an...,1609.07053v1,"We propose a novel semantic tagging task, semt...",0.437067
6,https://aclanthology.org/2021.woah-1.18,Community-level bans are a common tool against...,1610.01108v1,In this paper we provide the largest published...,0.144360
7,https://aclanthology.org/2021.wnut-1.1,Text simplification is the process of splittin...,1611.04798v1,"In this paper, we present our first attempts i...",0.442294
8,https://aclanthology.org/2021.wnut-1.3,We present the first openly available corpus f...,1610.04265v1,The utilization of statistical machine transla...,0.405618
9,https://aclanthology.org/2021.wnut-1.11,Multimodal named entity recognition (MNER) req...,1607.01628v1,"In this paper, we propose an effective way for...",0.509471


In [36]:
random.describe()

,index,cosine_abstract
count,10.00000,10.000000
mean,7914.50000,0.398246
std,3.02765,0.100333
min,7910.00000,0.144360
25%,7912.25000,0.399468
50%,7914.50000,0.427689
75%,7916.75000,0.444566
max,7919.00000,0.509471


In [40]:
random.columns

Index(['abstract', 'url', 'index', 'abstract_Arxiv', 'id_arxiv',
       'cosine_abstract'],
      dtype='object')

In [43]:
print(random[['url','abstract','id_arxiv', 'abstract_Arxiv','cosine_abstract']].round(2).to_latex(bold_rows=True))

\begin{tabular}{lllllr}
\toprule
{} &                                      url &                                           abstract &      id\_arxiv &                                     abstract\_Arxiv &  cosine\_abstract \\
\midrule
\textbf{0} &   https://aclanthology.org/2021.woah-1.2 &  Hate speech and profanity detection suffer fro... &  1607.08693v1 &  Although more additional corpora are now avail... &             0.45 \\
\textbf{1} &   https://aclanthology.org/2021.woah-1.3 &  We introduce HateBERT, a re-trained BERT model... &  1512.01100v1 &  Target-dependent sentiment classification rema... &             0.45 \\
\textbf{2} &   https://aclanthology.org/2021.woah-1.4 &  Hateful memes pose a unique challenge for curr... &  1607.00970v1 &  Using neural networks to generate replies in h... &             0.42 \\
\textbf{3} &   https://aclanthology.org/2021.woah-1.5 &  Content moderation is often performed by a col... &  1611.06639v1 &  Recurrent Neural Network (RNN) is one of the 

In [44]:
print(random['cosine_abstract'].describe().round(2).to_latex(bold_rows=True))

\begin{tabular}{lr}
\toprule
{} &  cosine\_abstract \\
\midrule
\textbf{count} &            10.00 \\
\textbf{mean } &             0.40 \\
\textbf{std  } &             0.10 \\
\textbf{min  } &             0.14 \\
\textbf{25\%  } &             0.40 \\
\textbf{50\%  } &             0.43 \\
\textbf{75\%  } &             0.44 \\
\textbf{max  } &             0.51 \\
\bottomrule
\end{tabular}

